In [1]:
print("Welcome to my EMR Notebook!")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
12,application_1597998755054_0027,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Welcome to my EMR Notebook!

In [2]:
# Import libraries
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, \
                                  avg, from_unixtime, split, min, max, round, lit, mean
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

#from pyspark.sql.types import IntegerType, TimestampType
import datetime
from pyspark.sql.functions import to_date, year, month, dayofmonth, dayofweek, hour, date_format, substring

import numpy as np
import time

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Create spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Set time parser policy
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Read in full sparkify dataset
event_data = "s3n://udacity-dsnd/sparkify/sparkify_event_data.json"
data = spark.read.json(event_data)
data.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

KeyboardInterrupt: 

### Logistic Regression

In [5]:
metrics = spark.read.json('s3n://amosvoron-sparkify/lr-metrics.json')
metrics.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------------------+------------------+------------------+------------------+
|               f1|          accuracy| weightedPrecision|    weightedRecall|               AUC|
+-----------------+------------------+------------------+------------------+------------------+
|0.774112692852762|0.8126263760952595|0.8012311289019549|0.8126263760952596|0.6116784733854458|
+-----------------+------------------+------------------+------------------+------------------+

### Random Forest

#### All 29 features (sparkify-random-forest.py)

In [4]:
metrics = spark.read.json('s3n://amosvoron-sparkify/rfc-metrics.json')
metrics.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------------+------------------+-----------------+------------------+
|                f1|         accuracy| weightedPrecision|   weightedRecall|               AUC|
+------------------+-----------------+------------------+-----------------+------------------+
|0.8305211981524745|0.853740732419681|0.8596862754487031|0.853740732419681|0.6889669702680066|
+------------------+-----------------+------------------+-----------------+------------------+

In [84]:
importances = spark.read.json('s3n://amosvoron-sparkify/rfc-feature-importances.json')
importances.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+
|feature|         importance|
+-------+-------------------+
|   col2| 0.5459124723013612|
|   col3|  0.254476479575686|
|   col1|0.19961104812295288|
+-------+-------------------+

#### 17 features, CV hyperparameters (sparkify-random-forest-2.py)

In [5]:
metrics = spark.read.json('s3n://amosvoron-sparkify/rfc-metrics.json')
metrics.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------------------+------------------+------------------+------------------+
|               f1|          accuracy| weightedPrecision|    weightedRecall|               AUC|
+-----------------+------------------+------------------+------------------+------------------+
|0.860977014640246|0.8723882273646372|0.8695028369486943|0.8723882273646372|0.7496408547366461|
+-----------------+------------------+------------------+------------------+------------------+

## Fine Tuning

In [ ]:
# Import libraries
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, \
                                  avg, from_unixtime, split, min, max, round, lit, mean
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

#from pyspark.sql.types import IntegerType, TimestampType
import datetime
from pyspark.sql.functions import to_date, year, month, dayofmonth, dayofweek, hour, date_format, substring

import numpy as np
import pandas as pd
import time
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sb

##### Functions

In [12]:

# Function that returns all users of a specified churn group
def get_users(churn):
    return data.where(data.churn == 1).select('userId').dropDuplicates()

# Count page logs
def page_count(page):
    return page_data \
        .where(data.page == page) \
        .groupby('userId') \
        .count()
        .select('userId', col('count').alias(page.replace(' ', '') + 'Count'))

# Average page count per session hour
def page_session_hour(page):
    return page_data \
        .where(data.page == page) \
        .join(session_hours, ['userId', 'sessionId'], 'inner') \
        .groupby( 'userId', 'sessionId', 'sessionHours') \
        .agg((count('userId')/col('sessionHours')).alias('avgPerSession')) \
        .groupby('userId') \
        .agg(avg('avgPerSession').alias('avg')) \
        .select('userId', col('avg').alias(page.replace(' ', '') + 'PerSessionHour'))

# Average page count per hour
def page_hour(page):
    return page_data \
        .where(data.page == page) \
        .join(user_hours, 'userId', 'inner') \
        .groupby('userId', 'hours') \
        .agg((count('userId')/col('hours')).alias('avg')) \
        .select('userId', col('avg').alias(page.replace(' ', '') + 'PerHour'))

# Average page count per day
def page_day(page):
    return page_data \
        .where(data.page == page) \
        .groupby('userId', 'date') \
        .count() \
        .groupby('userId') \
        .agg(avg('count').alias(page.replace(' ', '') + 'PerDay'))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### Prepare data

In [13]:

# Remove rows with missing users
data = data.where(~((col('userId').isNull()) | (col('userId') == '')))

# Exclude non-relevant columns
data = data.drop('firstName')
data = data.drop('lastName')

# Add tsDate and date column
data = data.withColumn('tsDate', (col('ts') / 1000.0).cast(TimestampType()))
data = data.withColumn('date', date_format(col('tsDate'), 'yyyy-MM-dd').alias('date').cast('date'))

# Label churned users using Cancellation Confirmation event
query_churn_by_cc = data.where(data.page == 'Cancellation Confirmation')
canceled = query_churn_by_cc.select('userId').dropDuplicates().select('userId')
canceled_uids = [row.userId for row in canceled.collect()];
set_churn = udf(lambda x: 1 if x in canceled_uids else 0, IntegerType())
data = data.withColumn('churn', set_churn('userId'))

# Add [userRowId] column that assigns a 1-based index to every user's log ordered by [ts]
w =  Window.partitionBy(data.userId).orderBy('ts', 'itemInSession')
data = data.withColumn('userRowId', row_number().over(w))

# Add [userRowDescId] column that assigns a 1-based index to every user's log ordered by [ts] descending.
w =  Window.partitionBy(data.userId).orderBy(col('ts').desc(), col('itemInSession').desc())
data = data.withColumn('userRowDescId', row_number().over(w))

# Add last level column
last_levels = dict()
for row in data.where(data.userRowDescId == 1).select('userId', 'level').collect():
    last_levels[row.userId] = row.level
get_level = udf(lambda userId: last_levels[userId])
data = data.withColumn('lastLevel', get_level('userId'))

# Prepare labels
labels = data.select(col('churn').alias('label'), 'userId').dropDuplicates()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- tsDate: timestamp (nullable = true)
 |-- date: date (nullable = true)
 |-- churn: integer (nullable = true)
 |-- userRowId: integer (nullable = true)
 |-- userRowDescId: integer (nullable = true)
 |-- lastLevel: string (nullable = true)

##### Queries

In [21]:

# All unique users
users = data.select('userId').dropDuplicates()

# Pages without churn definition events
page_data = data.where(~data.page.isin(['Cancel', 'Cancellation Confirmation'])) \
    .select('churn', 'page', 'userId', 'sessionId', 'ts', 'date')

# Calc session duration (in hours)
session_hours = page_data \
    .groupby('userId', 'sessionId') \
    .agg(((max('ts') - min('ts'))/1000/3600).alias('sessionHours'))

# User interactions duration per user (in hours)
user_hours = page_data \
    .groupby('userId', 'sessionId') \
    .agg(((max('ts') - min('ts'))/1000/3600).alias('sessionHours')) \
    .groupby('userId') \
    .agg(Fsum('sessionHours').alias('hours'))  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Features

In [16]:
    f_Gender = data \
        .select('userId', 'gender') \
        .dropDuplicates() \
        .replace(['M', 'F'], ['0', '1'], 'gender') \
        .select('userId', col('gender').cast('int').alias('Gender'))

    f_LastLevel = data \
        .select('userId', 'lastLevel') \
        .dropDuplicates() \
        .replace(['free', 'paid'], ['0', '1'], 'lastLevel') \
        .select('userId', col('lastLevel').cast('int').alias('LastLevel'))

    f_LogCount = data \
        .groupby('userId') \
        .agg(count('userId').alias('LogCount'))

    f_SongCount = data \
        .where(data.page == 'NextSong') \
        .groupby('userId') \
        .agg(count('userId').alias('SongCount'))

    f_NonSongCount = data \
        .where(data.page != 'NextSong') \
        .groupby('userId') \
        .agg(count('userId').alias('NonSongCount'))

    f_AboutCount = page_count('About')

    f_ThumbsUpCount = page_count('Thumbs Up')

    f_RollAdvertCount = page_count('Roll Advert')

    f_SessionCount = data \
        .select('userId', 'sessionId') \
        .dropDuplicates() \
        .groupby('userId') \
        .agg(count('userId').alias('SessionCount'))

    f_AvgSessionLength = data \
       .groupby('userId', 'sessionId') \
       .agg(((max('ts') - min('ts'))/1000).alias('sessionLength')) \
       .groupby('userId') \
       .agg(avg('sessionLength').alias('AvgSessionLength')) \

    f_AvgSessionGap = data \
        .groupby('userId', 'sessionId') \
        .agg(min('ts').alias('startTime'), max('ts').alias('endTime')) \
        .groupby('userId') \
        .agg(count('userId').alias('sessionCount'), \
            ((max('endTime') - min('startTime'))/1000).alias('observationPeriodTime'), \
            (Fsum(col('endTime') - col('startTime'))/1000).alias('totalSessionTime')) \
        .where(col('sessionCount') > 1) \
        .join(users, 'userId', 'outer') \
        .fillna(0) \
        .select('userId', \
            (col('observationPeriodTime') - col('totalSessionTime')/(col('sessionCount') - 1)).alias('AvgSessionGap'))

    f_DowngradePerSessionHour = page_session_hour('Downgrade')

    f_ErrorPerSessionHour = page_session_hour('Error')

    f_SettingsPerSessionHour = page_session_hour('Settings')

    f_SaveSettingsPerSessionHour = page_session_hour('Save Settings')

    f_LogoutPerSessionHour = page_session_hour('Logout')

    f_SubmitDowngradePerSessionHour = page_session_hour('Submit Downgrade')

    f_RollAdvertPerHour = page_hour('Roll Advert')

    f_ThumbsDownPerHour = page_hour('Thumbs Down')

    f_UpgradePerHour = page_hour('Upgrade')

    f_SubmitUpgradePerHour = page_hour('Submit Upgrade')

    f_SessionsPerDay = data \
        .select('userId', 'date', 'sessionId') \
        .dropDuplicates() \
        .groupby('userId', 'date') \
        .count() \
        .groupby('userId') \
        .agg(avg('count').alias('SessionsPerDay'))

    f_AddFriendPerDay = page_day('Add Friend')

    f_RollAdvertPerDay = page_day('Roll Advert')

    f_ThumbsDownPerDay = page_day('Thumbs Down')

    f_ThumbsUpPerDay = page_day('Thumbs Up')

    f_TotalSongLength = data \
        .where(data.page == 'NextSong') \
        .select('userId', 'length') \
        .groupby('userId') \
        .agg(Fsum('length').alias('TotalSongLength'))

    f_UniqueSongCount = data \
        .where(data.page == 'NextSong') \
        .select('userId', 'song') \
        .dropDuplicates() \
        .groupby('userId') \
        .agg(count('userId').alias('UniqueSongCount'))

    f_UniqueSongShare = data \
        .where(data.page == 'NextSong') \
        .select('userId', 'song') \
        .dropDuplicates() \
        .groupby('userId') \
        .count() \
        .join(f_SongCount, on = ['userId'], how = 'inner') \
        .select('userId', (col('count')/col('SongCount')).alias('UniqueSongShare')) 


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Feature Check

### AvgSessionGap (YES)

In [18]:
f_gaps = labels.join(f_AvgSessionGap, 'userId', 'outer') \
    .fillna(0)
f_gaps.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----+-----------------+
| userId|label|    AvgSessionGap|
+-------+-----+-----------------+
|1001393|    0|5037908.333333333|
|1002143|    0|4925333.714285715|
|1002493|    1|682813.6666666666|
|1002749|    0|4810390.533333333|
|1004060|    0|       4118678.76|
|1004316|    0|4581602.857142857|
|1006033|    0|        3844641.0|
|1006411|    0|5115310.333333333|
|1006697|    0|4368399.181818182|
|1008244|    1|         195181.0|
|1008404|    1|3341917.830188679|
|1010669|    0|        5150961.3|
|1010907|    0|4222507.285714285|
|1011093|    0|4962723.866666666|
|1011149|    0|4317323.444444444|
|1011630|    0|       2698736.25|
|1012906|    1|         852531.0|
|1013788|    0|        3225160.0|
|1017431|    0|         307951.0|
|1018499|    0|        4995090.5|
+-------+-----+-----------------+
only showing top 20 rows

In [20]:
f_gaps.groupby('label').agg(avg('AvgSessionGap')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------------------+
|label|avg(AvgSessionGap)|
+-----+------------------+
|    1| 2083145.432607491|
|    0| 3935744.507611506|
+-----+------------------+

### Per hour features

In [25]:
# Average page count per hour, per churn
page_data \
    .join(user_hours, 'userId', 'inner') \
    .groupby('churn', 'userId', 'page', 'hours') \
    .agg((count('userId')/col('hours')).alias('CountPerHour')) \
    .groupby('churn', 'page') \
    .agg(avg('CountPerHour').alias('AvgCountPerHour')) \
    .sort('page', 'churn') \
    .show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------------------+--------------------+
|churn|               page|     AvgCountPerHour|
+-----+-------------------+--------------------+
|    0|              About| 0.07056233416090642|
|    1|              About| 0.05558034658358316|
|    0|         Add Friend|  0.3217008363895395|
|    1|         Add Friend| 0.29844395045341365|
|    0|    Add to Playlist| 0.42611845979266155|
|    1|    Add to Playlist|  0.4215898833332168|
|    0|          Downgrade| 0.13972529087490962|
|    1|          Downgrade|  0.1859004628041162|
|    0|              Error|0.036940801089688724|
|    1|              Error| 0.03785066007418288|
|    0|               Help|  0.1135018729892504|
|    1|               Help| 0.11306976192257236|
|    0|               Home|  1.3388497396092534|
|    1|               Home|  0.8691192484357485|
|    0|              Login|0.020621423633006654|
|    0|             Logout|   1.450812997320228|
|    1|             Logout| 0.25510846892730715|
|    0|           Ne

##### Selected per hour features:

 - Downgrade
 - Home
 - Logout
 - Roll Advert
 - Settings
 - Submit Downgrade
 - Submit Upgrade
 - Thumbs Down

### Per day features

In [31]:
# Average page count per day, per churn
page_data \
    .groupby('churn', 'userId', 'page', 'date') \
    .count() \
    .groupby('churn', 'userId', 'page') \
    .agg(avg('count').alias('AvgCountPerDay')) \
    .groupby('churn', 'page') \
    .agg(avg('AvgCountPerDay').alias('AvgCountPerDay')) \
    .sort('page', 'churn') \
    .show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------------------+------------------+
|churn|               page|    AvgCountPerDay|
+-----+-------------------+------------------+
|    0|              About|1.2265077187557558|
|    1|              About|1.1872657560171735|
|    0|         Add Friend|2.4517891033084993|
|    1|         Add Friend| 2.564261041807439|
|    0|    Add to Playlist|2.6646704967460786|
|    1|    Add to Playlist| 2.827303473707138|
|    0|          Downgrade|1.8509720146725048|
|    1|          Downgrade| 1.940805990691478|
|    0|              Error|1.0762686323015083|
|    1|              Error|1.0822312859523107|
|    0|               Help|1.4402423265020787|
|    1|               Help|1.4400763045141485|
|    0|               Home|3.6046560619564607|
|    1|               Home|  3.59223269281641|
|    0|              Login| 4858.196721311476|
|    0|             Logout|1.6710914426251295|
|    1|             Logout|1.7796447995266915|
|    0|           NextSong|59.566890284449954|
|    1|      

##### Selected per day features:

 - RollAdvert

### SongCount (NO)

In [50]:
labels.join(f_SongCount, 'userId', 'outer') \
    .fillna(0) \
    .groupby('label').agg(avg('SongCount')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-----------------+
|label|   avg(SongCount)|
+-----+-----------------+
|    1|876.7291625024985|
|    0|953.0533140376266|
+-----+-----------------+

### NonSongCount (YES)

In [52]:
labels.join(f_NonSongCount, 'userId', 'outer') \
    .fillna(0) \
    .groupby('label').agg(avg('NonSongCount')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------------------+
|label| avg(NonSongCount)|
+-----+------------------+
|    1|199.11872876274236|
|    0|255.44057887120115|
+-----+------------------+

### SessionCount (YES)

In [53]:
labels.join(f_SessionCount, 'userId', 'outer') \
    .fillna(0) \
    .groupby('label').agg(avg('SessionCount')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------------------+
|label| avg(SessionCount)|
+-----+------------------+
|    1|12.300219868079152|
|    0|22.786685962373372|
+-----+------------------+

### AvgSessionLength (NO)

In [54]:
labels.join(f_AvgSessionLength, 'userId', 'outer') \
    .fillna(0) \
    .groupby('label').agg(avg('AvgSessionLength')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+---------------------+
|label|avg(AvgSessionLength)|
+-----+---------------------+
|    1|   17222.534302078067|
|    0|   16409.735186977854|
+-----+---------------------+

### TotalSongLength (NO)

In [55]:
labels.join(f_TotalSongLength, 'userId', 'outer') \
    .fillna(0) \
    .groupby('label').agg(avg('TotalSongLength')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+
|label|avg(TotalSongLength)|
+-----+--------------------+
|    1|   218025.0358112713|
|    0|  237060.12599282028|
+-----+--------------------+

### SessionsPerDay (NO)

In [56]:
labels.join(f_SessionsPerDay, 'userId', 'outer') \
    .fillna(0) \
    .groupby('label').agg(avg('SessionsPerDay')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------------------+
|label|avg(SessionsPerDay)|
+-----+-------------------+
|    1| 1.2193029303079534|
|    0| 1.3044363651611546|
+-----+-------------------+

### UniqueSongCount (NO)

In [57]:
labels.join(f_UniqueSongCount, 'userId', 'outer') \
    .fillna(0) \
    .groupby('label').agg(avg('UniqueSongCount')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+
|label|avg(UniqueSongCount)|
+-----+--------------------+
|    1|   779.5772536478113|
|    0|   842.7589001447178|
+-----+--------------------+

### UniqueSongShare (NO)

In [58]:
labels.join(f_UniqueSongShare, 'userId', 'outer') \
    .fillna(0) \
    .groupby('label').agg(avg('UniqueSongShare')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+
|label|avg(UniqueSongShare)|
+-----+--------------------+
|    1|  0.9428555327477975|
|    0|  0.9384634708370324|
+-----+--------------------+

### LogCount (NO)

In [59]:
labels.join(f_LogCount, 'userId', 'outer') \
    .fillna(0) \
    .groupby('label').agg(avg('LogCount')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------------------+
|label|     avg(LogCount)|
+-----+------------------+
|    1|1075.8478912652408|
|    0|1208.4938929088278|
+-----+------------------+

### AboutCount (YES)

In [60]:
labels.join(f_AboutCount, 'userId', 'outer') \
    .fillna(0) \
    .groupby('label').agg(avg('AboutCount')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-----------------+
|label|  avg(AboutCount)|
+-----+-----------------+
|    1|2.019188486907855|
|    0|4.784775687409551|
+-----+-----------------+

In [61]:
labels.join(f_ThumbsUpCount, 'userId', 'outer') \
    .fillna(0) \
    .groupby('label').agg(avg('ThumbsUpCount')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------------------+
|label|avg(ThumbsUpCount)|
+-----+------------------+
|    1|44.867479512292626|
|    0| 53.66095513748191|
+-----+------------------+

In [62]:
labels.join(f_RollAdvertCount, 'userId', 'outer') \
    .fillna(0) \
    .groupby('label').agg(avg('RollAdvertCount')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+
|label|avg(RollAdvertCount)|
+-----+--------------------+
|    1|   20.04917049770138|
|    0|  16.492387843704776|
+-----+--------------------+

### Per session-hour features

In [64]:
page_data \
    .join(session_hours, ['userId', 'sessionId'], 'inner') \
    .groupby('churn', 'page', 'userId', 'sessionId', 'sessionHours') \
    .agg((count('userId')/col('sessionHours')).alias('AvgPerSessionHour')) \
    .groupby('churn', 'page', 'userId') \
    .agg(avg('AvgPerSessionHour').alias('AvgPerSessionHour')) \
    .groupby('churn', 'page') \
    .agg(avg('AvgPerSessionHour').alias('AvgPerSessionHour')) \
    .sort('page', 'churn') \
    .show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------------------+-------------------+
|churn|               page|  AvgPerSessionHour|
+-----+-------------------+-------------------+
|    0|              About| 1.3286997598868076|
|    1|              About| 1.7558983615657775|
|    0|         Add Friend| 10.566364547227176|
|    1|         Add Friend| 10.577382252179353|
|    0|    Add to Playlist| 0.8948913425563265|
|    1|    Add to Playlist|  0.845031938914712|
|    0|          Downgrade| 0.5251358741020042|
|    1|          Downgrade| 0.9873323822777235|
|    0|              Error| 0.7684184077909724|
|    1|              Error| 0.3551135184536166|
|    0|               Help| 1.3482266412324133|
|    1|               Help| 1.1534599873795206|
|    0|               Home| 13.079747649035781|
|    1|               Home| 12.947641686686476|
|    0|              Login|  916.3842438628917|
|    0|             Logout| 22.457422711291965|
|    1|             Logout| 16.212140451544194|
|    0|           NextSong| 29.666904382

##### Selected per day features:

 - About
 - Downgrade
 - Error
 - Settings
 - Upgrade

### Page count features

In [71]:
page_data \
    .groupby('churn', 'page', 'userId') \
    .count() \
    .groupby('churn', 'page') \
    .agg(avg('count').alias('AvgPageCount')) \
    .sort('page', 'churn') \
    .show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------------------+------------------+
|churn|               page|      AvgPageCount|
+-----+-------------------+------------------+
|    0|              About| 7.283196757423561|
|    1|              About|3.2461439588688945|
|    0|         Add Friend| 19.09927820691402|
|    1|         Add Friend|17.736643759698516|
|    0|    Add to Playlist|28.653205787976024|
|    1|    Add to Playlist|26.282099936748892|
|    0|          Downgrade|12.295119182746879|
|    1|          Downgrade|11.561235356762513|
|    0|              Error|2.3408214204674382|
|    1|              Error| 2.167547783651891|
|    0|               Help| 8.957388557388557|
|    1|               Help| 6.877219304826206|
|    0|               Home| 66.83783626072241|
|    1|               Home|  40.0530794165316|
|    0|              Login|          296350.0|
|    0|             Logout|14.109767160568492|
|    1|             Logout|13.556756756756757|
|    0|           NextSong| 953.9368445448752|
|    1|      

In [77]:
55/(55+46.5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.541871921182266

### ML - Random Forest Test

In [78]:
# Import ML libraries
from pyspark.ml.feature import VectorAssembler, Normalizer, StandardScaler, MinMaxScaler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
import numpy as np

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
columns = ['label', 'col1', 'col2', 'col3']
values = [[0, 0, 4, 7], [0, 0, 50, 19], [0, 0, 49, 2], [0, 0, 51, 6], [0, 1, 50, 99], \
          [1, 1, 22, 77], [1, 1, 97, 13], [1, 1, 89, 4], [1, 1, 98, 39], [1, 0, 93, 12],
          [1, 1, 101, 7], [1, 1, 105, 53], [1, 1, 96, 67], [1, 1, 100, 81], [1, 0, 105, 2]]
rdd = spark.sparkContext.parallelize(values)
df = rdd.toDF(columns) 
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----+----+----+
|label|col1|col2|col3|
+-----+----+----+----+
|    0|   0|   4|   7|
|    0|   0|  50|  19|
|    0|   0|  49|   2|
|    0|   0|  51|   6|
|    0|   1|  50|  99|
|    1|   1|  22|  77|
|    1|   1|  97|  13|
|    1|   1|  89|   4|
|    1|   1|  98|  39|
|    1|   0|  93|  12|
|    1|   1| 101|   7|
|    1|   1| 105|  53|
|    1|   1|  96|  67|
|    1|   1| 100|  81|
|    1|   0| 105|   2|
+-----+----+----+----+

In [80]:
# Return selected features without label  
def get_features(data):
    return data.drop('label').columns

# Split data into training and testing subset
def split_data(data, seed=0):
    train, test = data.randomSplit([0.8, 0.2], seed=seed);
    return train, test

# Create ML pipeline
def create_pipeline(data, classifier, scaler):
    features = get_features(data)
    assembler = VectorAssembler(inputCols=features, outputCol='NumFeatures')
    pipeline = Pipeline(stages=[assembler, scaler, classifier]);
    return pipeline

# Fit and evaluate model
def fit_model(data, classifier, seed=0):
    train, test = split_data(data, seed)
    scaler = StandardScaler(inputCol='NumFeatures', outputCol='features')
    pipeline = create_pipeline(data, classifier, scaler)
    model = pipeline.fit(train)
    return model

def calc_metrics(model, test):
    # Create both evaluators
    evaluatorMulti = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction')
    evaluator = BinaryClassificationEvaluator(labelCol='label', rawPredictionCol='prediction', metricName='areaUnderROC')

    # Make predicitons
    prediction = model.transform(test).select('label', 'prediction')

    # Get metrics
    accuracy = evaluatorMulti.evaluate(prediction, {evaluatorMulti.metricName: 'accuracy'})
    f1 = evaluatorMulti.evaluate(prediction, {evaluatorMulti.metricName: 'f1'})
    weightedPrecision = evaluatorMulti.evaluate(prediction, {evaluatorMulti.metricName: 'weightedPrecision'})
    weightedRecall = evaluatorMulti.evaluate(prediction, {evaluatorMulti.metricName: 'weightedRecall'})
    auc = evaluator.evaluate(prediction)

    # Write metrics to S3
    columns = ['f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC']
    values = [f1, accuracy, weightedPrecision, weightedRecall, auc]
    rdd = spark.sparkContext.parallelize([values])
    metrics = rdd.toDF(columns)
    
    return metrics

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
def get_feature_importances()

In [81]:
classifier = RandomForestClassifier(seed=0)
model = fit_model(df, classifier)
train, test = split_data(df)
metrics = calc_metrics(model, test)
metrics.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------+------------------+--------------+----+
|                f1|accuracy| weightedPrecision|weightedRecall| AUC|
+------------------+--------+------------------+--------------+----+
|0.7333333333333334|    0.75|0.8333333333333333|          0.75|0.75|
+------------------+--------+------------------+--------------+----+

In [82]:
values = []
features = list(get_features(df))
importances = list(model.stages[-1].featureImportances)
for x in list(zip(features, importances)):
    values.append([x[0], float(x[1])])
columns = ['feature', 'importance']
rdd = spark.sparkContext.parallelize(values)
df2 = rdd.toDF(columns) 
df2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+
|feature|         importance|
+-------+-------------------+
|   col1|0.19961104812295288|
|   col2| 0.5459124723013612|
|   col3|  0.254476479575686|
+-------+-------------------+

In [83]:
df2.write.mode('overwrite').json('s3n://amosvoron-sparkify/rfc-feature-importances-test.json')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Users

In [87]:
get_users(1).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5003

In [88]:
get_users(0).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '404' from https://172.31.39.107:18888/sessions/8/statements/88 with error payload: {"msg":"Session '8' not found."}


In [1]:
data.where(data.page == 'Submit Downgrade').select('userId').dropDuplicates().count()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
11,application_1597998755054_0026,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'data' is not defined
Traceback (most recent call last):
NameError: name 'data' is not defined



# Check metrics & feature importances

In [4]:
metrics = spark.read.json('s3n://amosvoron-sparkify/rfc-metrics-f22.json')
metrics.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+-----------------+------------------+------------------+
|                f1|          accuracy|weightedPrecision|    weightedRecall|               AUC|
+------------------+------------------+-----------------+------------------+------------------+
|0.8386651060216977|0.8577847674679847|0.858875141192138|0.8577847674679848|0.7055267657733111|
+------------------+------------------+-----------------+------------------+------------------+

In [9]:
importances = spark.read.json('s3n://amosvoron-sparkify/rfc-feature-importances-f22.json')
importances.sort(desc('importance')).show(30, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+---------------------+
|feature                |importance           |
+-----------------------+---------------------+
|AvgSessionGap          |0.531326926781188    |
|RollAdvertPerHour      |0.07203686762635017  |
|DowngradePerHour       |0.06676894742434188  |
|SessionCount           |0.062376199033023286 |
|NonSongCount           |0.050628344100157416 |
|RollAdvertPerDay       |0.04571131362963781  |
|RollAdvertCount        |0.03022773918358667  |
|DowngradePerSessionHour|0.023059199682565503 |
|ThumbsDownPerHour      |0.021905197950220757 |
|HomeCount              |0.01935741525646098  |
|LogoutPerHour          |0.014225091004298893 |
|ThumbsUpCount          |0.014185526067462045 |
|SettingsPerSessionHour |0.011980284038583023 |
|AboutCount             |0.00900477412830605  |
|SubmitUpgradePerHour   |0.006285279375803763 |
|SettingsPerHour        |0.005847270443837536 |
|SubmitDowngradePerHour |0.004204600333886947 |
|UpgradePerSessionHour  |0.0030743231489

In [ ]:
metrics = spark.read.json('s3n://amosvoron-sparkify/rfc-metrics-f14.json')
metrics.select('f1', 'accuracy', 'weightedPrecision', 'weightedRecall', 'AUC').show()

In [ ]:
importances = spark.read.json('s3n://amosvoron-sparkify/rfc-feature-importances-f14.json')
importances.sort(desc('importance')).show(30, False)